In [3]:
from google.colab import files
uploades = files.upload()

Saving peliculas_enriquecidas.csv to peliculas_enriquecidas.csv


In [23]:
import pandas as pd

df = pd.read_csv('peliculas_enriquecidas.csv')
df = df.dropna(subset=['overview_lemmatized', 'title'])
df = df.reset_index(drop=True)
df.head()

,id,title,original_title,production_countries,overview,release_date,original_language,genres,runtime,budget,...,cast,vote_average,director,keywords,main_country,overview_tokens,overview_lemmas,overview_lemmatized,keywords_tfidf,topic_id
0,567609,Noche de bodas,Ready or Not,['United States of America'],"Durante la noche de su boda, una mujer recibe ...",2019-08-21,en,"['Terror', 'Comedia']",95,6000000,...,"['Samara Weaving', 'Adam Brody', ""Mark O'Brien""]",7.0,Tyler Gillett,"['escape', 'ritual', 'marriage', 'satanism', '...",United States of America,"['noche', 'boda', 'mujer', 'recibe', 'invitaci...","['noche', 'bodo', 'mujer', 'recibir', 'invitac...",noche bodo mujer recibir invitación parte rico...,"['supervivencia', 'ancestral', 'tradición', 'l...",7
1,299534,Vengadores: Endgame,Avengers: Endgame,['United States of America'],Después de los eventos devastadores de 'Vengad...,2019-04-24,en,"['Aventura', 'Ciencia ficción', 'Acción']",181,356000000,...,"['Robert Downey Jr.', 'Chris Evans', 'Mark Ruf...",8.2,Anthony Russo,"['superhero', 'time travel', 'space travel', '...",United States of America,"['después', 'eventos', 'devastadores', 'vengad...","['después', 'eventos', 'devastador', 'vengador...",después eventos devastador vengador infinity w...,"['vengador', 'universo', 'acción', 'vez', 'ruin']",5
2,299537,Capitana Marvel,Captain Marvel,['United States of America'],La historia sigue a Carol Danvers mientras se ...,2019-03-06,en,"['Acción', 'Aventura', 'Ciencia ficción']",125,152000000,...,"['Brie Larson', 'Samuel L. Jackson', 'Ben Mend...",6.8,Ryan Fleck,"['superhero', 'based on comic', 'alien invasio...",United States of America,"['historia', 'sigue', 'carol', 'danvers', 'mie...","['historia', 'seguir', 'carol', 'danvers', 'mi...",historia seguir carol danvers mientras convert...,"['universo', 'historia', 'período', 'cinematog...",5
3,320288,X-Men: Fénix oscura,Dark Phoenix,['United States of America'],Los X-Men se enfrentan a su enemigo más podero...,2019-06-05,en,"['Ciencia ficción', 'Acción', 'Aventura']",116,200000000,...,"['Sophie Turner', 'James McAvoy', 'Nicholas Ho...",6.0,Simon Kinberg,"['superhero', 'mutant', 'telekinesis', 'based ...",United States of America,"['men', 'enfrentan', 'enemigo', 'poderoso', 'm...","['men', 'enfrentar', 'enemigo', 'poderoso', 'm...",men enfrentar enemigo poderoso miembro jear gr...,"['jean', 'poderoso', 'lazo', 'daño', 'espiral']",5
4,515001,Jojo Rabbit,Jojo Rabbit,"['New Zealand', 'United States of America']",Jojo Rabbit es un niño viviendo en plena 2ª Gu...,2019-10-18,en,"['Comedia', 'Bélica', 'Drama']",108,14000000,...,"['Roman Griffin Davis', 'Thomasin McKenzie', '...",8.0,Taika Waititi,"['based on novel or book', 'world war ii', 'je...",New Zealand,"['jojo', 'rabbit', 'niño', 'viviendo', 'plena'...","['jojo', 'rabbit', 'niño', 'vivir', 'pleno', '...",jojo rabbit niño vivir pleno guerra mundial ún...,"['niño', 'escape', 'incitar', 'hitler', 'vía']",8


In [24]:
# Vectorizamos cada campo por separado

from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
# Overview

tfidf_overview = TfidfVectorizer(max_features=5000)
X_overview = tfidf_overview.fit_transform(df['overview_lemmatized'].astype(str))

In [26]:
# keywords

tfidf_keywords = TfidfVectorizer()
X_keywords = tfidf_keywords.fit_transform(df['keywords_tfidf'].astype(str))

In [27]:
# Genres

tfidf_genres = TfidfVectorizer()
X_genres = tfidf_genres.fit_transform(df['genres'].astype(str))

In [28]:
# Director

tfidf_director = TfidfVectorizer()
X_director = tfidf_director.fit_transform(df['director'].astype(str))

In [29]:
# TopicID

df['topic_id_str'] = df['topic_id'].apply(lambda x: f"topic_{int(x)}")
tfidf_topic = TfidfVectorizer()
X_topic = tfidf_topic.fit_transform(df['topic_id_str'])

In [30]:
# País de producción

tfidf_country = TfidfVectorizer()
X_country = tfidf_country.fit_transform(df['production_countries'].astype(str))

In [31]:
# Normalizamos y combinamos matrices TF-IDF con pesos

from sklearn.preprocessing import normalize
from scipy.sparse import hstack

# Normalizamos
X_overview_n = normalize(X_overview)
X_keywords_n = normalize(X_keywords)
X_genres_n = normalize(X_genres)
X_director_n = normalize(X_director)
X_country_n = normalize(X_country)
X_topic_n = normalize(X_topic)

# Aplicamos pesos
X_overview_w = 0.20 * X_overview_n
X_keywords_w = 0.28 * X_keywords_n
X_genres_w = 0.20 * X_genres_n
X_director_w = 0.26 * X_director_n
X_country_w = 0.005 * X_country_n
X_topic_w = 0.055 * X_topic_n

# Concatenamos horizontalmente
X_combined = hstack([
    X_overview_w,
    X_keywords_w,
    X_genres_w,
    X_director_w,
    X_country_w,
    X_topic_w
])

In [32]:
# Calculamos la similitud

from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(X_combined)

In [33]:
# Función de recomendación

def recomendar_peliculas(titulo_pelicula, df, X_overview_w, X_keywords_w, X_genres_w, X_director_w, X_country_w, X_topic_w, top_n=5):
  if titulo_pelicula not in df['title'].values:
    print("Pelicula no encontrada.")
    return

  idx = df[df['title'] == titulo_pelicula].index[0]

  # Calcular similitudes individuales
  sim_overview = cosine_similarity(X_overview_w[idx], X_overview_w).flatten()
  sim_keywords = cosine_similarity(X_keywords_w[idx], X_keywords_w).flatten()
  sim_genres = cosine_similarity(X_genres_w[idx], X_genres_w).flatten()
  sim_director = cosine_similarity(X_director_w[idx], X_director_w).flatten()
  sim_country = cosine_similarity(X_country_w[idx], X_country_w).flatten()
  sim_topic = cosine_similarity(X_topic_w[idx], X_topic_w).flatten()

  # Sumamos similitudes ponderadas
  sim_total = sim_overview + sim_keywords + sim_genres + sim_director + sim_country + sim_topic

  #Creamos dataframe temporal con los resultados
  df_sim = df.copy()
  df_sim['similitud'] = sim_total

  # Eliminamos película base de los resultados
  df_sim = df_sim.drop(idx)

  # Ordenamos por similitud
  df_sim = df_sim.sort_values(by='similitud', ascending= False)

  # Eliminamos duplicados
  df_sim = df_sim.drop_duplicates(subset='id')

  recomendaciones = df_sim.head(top_n)

  # Obtenemos recomendaciones
  #recomendaciones = df_sim.sort_values(by='similitud', ascending=False).head(top_n)

  print(f"\nPelícula base: {titulo_pelicula}\nRecomendaciones:\n")

  for i, row in recomendaciones.iterrows():
    contribuciones = {
        'overview': sim_overview[i],
        'keywords': sim_keywords[i],
        'genres': sim_genres[i],
        'director': sim_director[i],
        'country': sim_country[i],
        'topic': sim_topic[i]

    }

    razones = sorted(contribuciones.items(), key=lambda x: x[1], reverse=True)
    razones_utiles = [r[0] for r in razones if r[1] > 0.01][:2]

    print(f"🎬 {row['title']} (ID: {row['id']}) - Similitud: {row['similitud']:.3f} - Razones: {razones_utiles}")




In [34]:
# Prueba

recomendar_peliculas(
    'Constantine',
    df,
    X_overview_w,
    X_keywords_w,
    X_genres_w,
    X_director_w,
    X_country_w,
    X_topic_w
)


Película base: Constantine
Recomendaciones:

🎬 El fin de los días (ID: 9946) - Similitud: 2.638 - Razones: ['genres', 'topic']
🎬 Soy leyenda (ID: 6479) - Similitud: 2.620 - Razones: ['topic', 'director']
🎬 Los 4 fantásticos (ID: 9738) - Similitud: 2.555 - Razones: ['country', 'topic']
🎬 Doctor Sueño (ID: 501170) - Similitud: 2.501 - Razones: ['topic', 'genres']
🎬 Entrevista con el vampiro (ID: 628) - Similitud: 2.461 - Razones: ['topic', 'genres']


In [35]:
recomendar_peliculas(
    'Los aristogatos',
    df,
    X_overview_w,
    X_keywords_w,
    X_genres_w,
    X_director_w,
    X_country_w,
    X_topic_w
)


Película base: Los aristogatos
Recomendaciones:

🎬 Tom y Jerry: Rumbo a marte (ID: 21078) - Similitud: 3.102 - Razones: ['country', 'topic']
🎬 El alucinante mundo de Norman (ID: 77174) - Similitud: 3.041 - Razones: ['genres', 'country']
🎬 Mascotas 2 (ID: 412117) - Similitud: 3.031 - Razones: ['genres', 'country']
🎬 Al rescate de Fondo de Bikini: La película de Arenita Mejillas (ID: 831815) - Similitud: 3.013 - Razones: ['genres', 'country']
🎬 Stuart Little: Aventura en el bosque (ID: 30178) - Similitud: 3.003 - Razones: ['country', 'topic']


In [36]:
recomendar_peliculas(
    'Delta Force 2',
    df,
    X_overview_w,
    X_keywords_w,
    X_genres_w,
    X_director_w,
    X_country_w,
    X_topic_w
)


Película base: Delta Force 2
Recomendaciones:

🎬 Navy Seals, comando especial (ID: 427910) - Similitud: 3.066 - Razones: ['genres', 'country']
🎬 Amanecer rojo (ID: 60747) - Similitud: 2.755 - Razones: ['country', 'topic']
🎬 Pootie Tang (ID: 10615) - Similitud: 2.749 - Razones: ['country', 'topic']
🎬 Y2K (ID: 17122) - Similitud: 2.747 - Razones: ['country', 'topic']
🎬 Triple frontera (ID: 399361) - Similitud: 2.732 - Razones: ['country', 'topic']


In [37]:
recomendar_peliculas(
    'Wolfs',
    df,
    X_overview_w,
    X_keywords_w,
    X_genres_w,
    X_director_w,
    X_country_w,
    X_topic_w
)

Pelicula no encontrada.


In [38]:
recomendar_peliculas(
    'Garfield: La película',
    df,
    X_overview_w,
    X_keywords_w,
    X_genres_w,
    X_director_w,
    X_country_w,
    X_topic_w
)


Película base: Garfield: La película
Recomendaciones:

🎬 TMNT: Tortugas Ninja Jóvenes Mutantes (ID: 1273) - Similitud: 2.930 - Razones: ['genres', 'topic']
🎬 Madagascar (ID: 953) - Similitud: 2.518 - Razones: ['genres', 'topic']
🎬 El último unicornio (ID: 10150) - Similitud: 2.437 - Razones: ['topic', 'genres']
🎬 Valiant (ID: 14175) - Similitud: 2.435 - Razones: ['topic', 'genres']
🎬 Pinocho de Guillermo del Toro (ID: 555604) - Similitud: 2.421 - Razones: ['topic', 'genres']


In [39]:
recomendar_peliculas(
    'Eva al desnudo',
    df,
    X_overview_w,
    X_keywords_w,
    X_genres_w,
    X_director_w,
    X_country_w,
    X_topic_w
)


Película base: Eva al desnudo
Recomendaciones:

🎬 Como plaga de langosta (ID: 5122) - Similitud: 3.119 - Razones: ['genres', 'country']
🎬 Sombras (ID: 15484) - Similitud: 3.081 - Razones: ['genres', 'country']
🎬 La gran esperanza blanca (ID: 17978) - Similitud: 3.059 - Razones: ['genres', 'country']
🎬 Vuelve a la tienda de baratijas, Jimmy Dean (ID: 67772) - Similitud: 3.047 - Razones: ['genres', 'country']
🎬 Hitchcock (ID: 112336) - Similitud: 3.037 - Razones: ['genres', 'country']


In [40]:
recomendar_peliculas(
    'Patton',
    df,
    X_overview_w,
    X_keywords_w,
    X_genres_w,
    X_director_w,
    X_country_w,
    X_topic_w
)


Película base: Patton
Recomendaciones:

🎬 El Batallón perdido (ID: 35583) - Similitud: 2.838 - Razones: ['country', 'topic']
🎬 Arenas Sangrientas (ID: 18712) - Similitud: 2.828 - Razones: ['country', 'topic']
🎬 Los tigres del mar (ID: 99345) - Similitud: 2.821 - Razones: ['country', 'topic']
🎬 Día D, hora H (ID: 60908) - Similitud: 2.770 - Razones: ['country', 'topic']
🎬 Éxodo (ID: 1941) - Similitud: 2.754 - Razones: ['country', 'topic']


In [41]:
recomendar_peliculas(
    'El especialista',
    df,
    X_overview_w,
    X_keywords_w,
    X_genres_w,
    X_director_w,
    X_country_w,
    X_topic_w
)


Película base: El especialista
Recomendaciones:

🎬 El agente invisible (ID: 725201) - Similitud: 3.311 - Razones: ['country', 'topic']
🎬 El invitado (ID: 59961) - Similitud: 3.230 - Razones: ['country', 'topic']
🎬 Enemy Gold (ID: 49983) - Similitud: 3.017 - Razones: ['country', 'topic']
🎬 Speed: Máxima potencia (ID: 1637) - Similitud: 3.000 - Razones: ['country', 'topic']
🎬 La última fortaleza (ID: 2100) - Similitud: 2.890 - Razones: ['country', 'topic']


In [22]:
from collections import Counter

all_genres = df['genres'].explode()
genre_counts = Counter(all_genres)

for genre, count in genre_counts.most_common():
    print(f"{genre}: {count}")

['Drama']: 312
['Comedia']: 212
['Drama', 'Romance']: 134
['Comedia', 'Romance']: 96
['Comedia', 'Drama']: 89
['Comedia', 'Drama', 'Romance']: 69
['Western']: 65
['Terror', 'Suspense']: 62
['Terror']: 60
['Drama', 'Suspense']: 54
['Romance', 'Drama']: 50
['Crimen', 'Drama']: 47
['Acción', 'Suspense']: 42
['Drama', 'Comedia']: 40
['Crimen', 'Drama', 'Suspense']: 37
['Documental']: 37
['Acción']: 36
['Drama', 'Historia']: 34
['Comedia', 'Crimen']: 33
['Drama', 'Película de TV']: 31
['Acción', 'Comedia']: 30
['Drama', 'Crimen']: 27
['Acción', 'Crimen', 'Suspense']: 26
['Acción', 'Crimen', 'Drama']: 26
[]: 26
['Acción', 'Crimen']: 25
['Suspense']: 25
['Acción', 'Comedia', 'Crimen']: 24
['Terror', 'Ciencia ficción']: 21
['Terror', 'Misterio']: 21
['Comedia', 'Familia']: 21
['Acción', 'Drama']: 20
['Drama', 'Comedia', 'Romance']: 20
['Acción', 'Aventura', 'Ciencia ficción']: 19
['Crimen']: 19
['Acción', 'Suspense', 'Crimen']: 18
['Terror', 'Misterio', 'Suspense']: 18
['Romance', 'Comedia']: 